<a href="https://colab.research.google.com/github/codesamskaaraH/ML_dev/blob/master/Pumpitup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
X_train_df = pd.read_csv("/content/drive/My Drive/DrivenData/pumpitup/TrainValues.csv", index_col='id')
Y_train_df = pd.read_csv("/content/drive/My Drive/DrivenData/pumpitup/Training_labels.csv", index_col= 'id')
#index_col is used as row label for this data set as the each building is uniquely identified by this id, also
#submission CSV requires this id

In [5]:
#various prints to check the data set
#print(X_train_df.describe())
print(X_train_df.info())
# shape of df is = 260601, 38
print(X_train_df.shape)
#X_train_df.describe()
#X_train_df.dtypes
X_train_df.head(5)
#print(Y_train_df.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [0]:
#discriptions
#functional - the waterpoint is operational and there are no repairs needed
#functional needs repair - the waterpoint is operational, but needs repairs
#non functional - the waterpoint is not operational

#clean up
features_may_not_be_needed = ["funder", "wpt_name", "num_private", "basin", 
                              "subvillage", "region", "region_code", "district_code", 
                              "lga","ward", "recorded_by", "scheme_management",
                              "scheme_name", "permit", "extraction_type", 
                              "extraction_type_group", "management", "management_group",
                              "payment", "payment_type", "water_quality", "quality_group",
                              "source", "source_type"]
X_parishkratam_df = X_train_df.drop(columns=features_may_not_be_needed)

In [7]:
X_parishkratam_df.shape

(59400, 15)

In [0]:
dummy_nonnumeric_cols = X_parishkratam_df.select_dtypes('O').columns.to_list()


In [9]:
dummy_nonnumeric_cols

['date_recorded',
 'installer',
 'public_meeting',
 'extraction_type_class',
 'quantity',
 'quantity_group',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [10]:
Y_train_df

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional
...,...
60739,functional
27263,functional
37057,functional


In [0]:
dummy_nonnumeric_cols = X_parishkratam_df.select_dtypes('O').columns.to_list()
X_train_numerics = pd.concat([X_parishkratam_df.drop(columns=dummy_nonnumeric_cols),
           pd.get_dummies(X_parishkratam_df[dummy_nonnumeric_cols]),
          ], axis=1)


In [12]:
X_train_numerics.shape

(59400, 2542)

In [14]:

X_train_numerics
X_train_numerics.shape

(59400, 2542)

In [45]:
# Prepare test data, this was added here for the reason that sometimes train and test data set rows needs to be normalized
#
#one of the check could be to find out whether train columns matches with test columns, if not train columns needs to be adjusted
#accordingly, in this data set that is what happened
X_test_df = pd.read_csv("/content/drive/My Drive/DrivenData/pumpitup/Testset_values.csv",  index_col= 'id')

X_test_df.head
X_test_df.shape

(14850, 39)

In [46]:
X_test_parishkratam_df = X_test_df.drop(columns=features_may_not_be_needed)
X_test_parishkratam_df.shape

(14850, 15)

In [47]:

dummy_nonnumeric_cols_test = X_test_parishkratam_df.select_dtypes('O').columns.to_list()
dummy_nonnumeric_cols_test


['date_recorded',
 'installer',
 'public_meeting',
 'extraction_type_class',
 'quantity',
 'quantity_group',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [0]:
X_test_numerics = pd.concat([X_test_parishkratam_df.drop(columns=dummy_nonnumeric_cols_test),
           pd.get_dummies(X_test_parishkratam_df[dummy_nonnumeric_cols_test]),
          ], axis=1)

In [51]:
print(X_test_numerics.shape)
print(X_train_numerics.shape)


(14850, 1463)
(59400, 2542)


In [0]:
#rows mismtach here between test and train, so normalize test DF to match train DF, well, there could be data loss
final_train, final_test = X_train_numerics.align(X_test_numerics, join='inner', axis=1)  # inner join




In [54]:
final_train.shape, final_test.shape

((59400, 1185), (14850, 1185))

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [56]:
print(Y_train_df.head)

<bound method NDFrame.head of          status_group
id                   
69572      functional
8776       functional
34310      functional
67743  non functional
19728      functional
...               ...
60739      functional
27263      functional
37057      functional
31282      functional
26348      functional

[59400 rows x 1 columns]>


In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(final_train, Y_train_df, test_size=0.2)





In [58]:
y_train.head()

,status_group
id,
58389,non functional
73398,non functional
42368,non functional
35819,non functional
2679,functional


In [59]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((47520, 1185), (11880, 1185), (47520, 1), (11880, 1))

In [0]:
model_pumpit = LogisticRegression(solver="liblinear")
#model_eq = RandomForestClassifier(min_samples_leaf=10,  n_estimators=100, n_jobs=-1)





In [61]:
model_pumpit.fit(X_train, y_train["status_group"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
def compute_accuracy(model, X, y):
    y_pred = model.predict(X)
    print(confusion_matrix(y, y_pred))
    return accuracy_score(y, y_pred), f1_score(y, y_pred, average='micro')

In [63]:
compute_accuracy(model_pumpit, X_train, y_train["status_group"])

[[23649    40  2136]
 [ 2870    25   561]
 [ 7136    45 11058]]


(0.7308922558922559, 0.7308922558922559)

In [0]:
predictions = model_pumpit.predict(final_test)

In [67]:
predictions

array(['non functional', 'functional', 'non functional', ...,
       'functional', 'functional', 'non functional'], dtype=object)

In [0]:
#submission

In [0]:
submission_01= pd.read_csv('/content/drive/My Drive/DrivenData/pumpitup/SubmissionFormat.csv', index_col='id')

In [0]:
my_submission_01 = pd.DataFrame(data=predictions,
                                columns =  submission_01.columns,
                                index = submission_01.index)

In [71]:
my_submission_01.head

<bound method NDFrame.head of          status_group
id                   
50785  non functional
51630      functional
17168  non functional
45559  non functional
49871      functional
...               ...
39307      functional
18990      functional
28749      functional
33492      functional
68707  non functional

[14850 rows x 1 columns]>

In [0]:
my_submission_01.to_csv('/content/drive/My Drive/DrivenData/pumpitup/submission_pup_001.csv')
